**извлечение признаков из текста на естественном языке**

класстеризатор текстов

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [3]:
pd.options.display.max_colwidth = 200  

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))

записей: 3196


In [5]:
data.sample(2)

,text,tag
2639,"Ученые выяснили сколько нужно спать, чтобы быть бодрым весь день\n\nИсследователи из Финляндии определили оптимальное количество времени для сна. Они провели ряд экспериментов, в которых приняли у...",science
1559,"Российский Центр примирения сторон продолжает оказывать помощь населению Восточного Алеппо. За последние сутки из этой части города эвакуировали 8 тысяч жителей, из них почти четыре тысячи — дети....",politics


## токенизация и очистка

In [6]:
from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [7]:
def preprocessor(text):
    tt = [ t for t in text.split() if t ]
    tt = [ t.lower()  for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
    tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
    tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
    tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ t for t in tt if t not in ['в','на','за','для','под','над'] ] # удаление (предлогов)
    tt = Stemmer('russian').stemWords( tt ) # стемминг, выделение основы слова
    tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
    tt = [ t for t in tt if len(t)>2 ] # удаление коротких слов (предлогов)
    return ' '.join( [ t.strip() for t in tt if t ] )
    

##  CountVectorizer + TF-IDF

In [8]:
# TfidfVectorizer = CountVectorizer + TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(preprocessor=preprocessor)
tf.fit( data['text'] )

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preprocessor at 0x7fb315c5b510>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=True, vocabulary=None)

In [9]:
# размер словаря
len(tf.vocabulary_)

37661

## формируем датасеты

In [10]:
X = tf.transform( data['text'] ) # .todense()
X.shape

(3196, 37661)

## кластеризируем

In [14]:
from time import time
def get_seed(): t = time() ; return int(((t%1)/(t//1))*1e11)

In [31]:
%%time

n_clusters = len(set(data['tag']))
kmeans = KMeans(n_clusters=n_clusters, random_state=get_seed() )
kmeans.fit(X)

CPU times: user 1min, sys: 46.5 ms, total: 1min
Wall time: 52.2 s


In [34]:
data['cluster'] = kmeans.labels_

## тестируем

In [35]:
data[ data['cluster']==0 ].sample(10)

,text,tag,cluster
2214,"В поезде Москва — Киев, который должен был отправиться в столицу Украины утром 13 декабря, полицейские и кинологи ищут взрывное устройство, о наличии которого по телефону сообщил неизвестный.\n\nО...",incident,0
1692,"После посадки в аэропорту Нью-Йорка воздушное судно отогнали в отдаленную часть аэропорта, где прошла эвакуация пассажиров и членов экипажа. Самолет немецкой авиакомпании экстренно приземлился в м...",incident,0
1754,"Это не повлияет сильно на финансовые результаты этого года, уверены в компании.\n\nАмериканская корпорация Boeing объявила о планах по сокращению производства самолетов Boeing 777 на 40% в 2017 го...",economics,0
2216,"КИШИНЕВ, 13 дек — Sputnik. Силовики во вторник утром проверяют поезд сообщением Москва — Киев на Киевском вокзале в центре Москвы после анонимного сообщения о взрывном устройстве, сообщает РИА Нов...",incident,0
2213,"Во вторник утром на Киевском вокзале в центре Москвы спецслужбы проверяют поезд Москва — Киев после анонимного сообщения о взрывном устройстве, сообщает РИА Новости.\n\nПо данным экстренных служб ...",incident,0
1689,"Самолёт немецкой авиакомпании Lufthansa, следовавший из Хьюстона, совершил экстренную посадку в аэропорту Нью-Йорка. Об этом сообщает New York Daily News.\n\nПо данным издания, Airbus 380 совершил...",incident,0
2219,"МОСКВА, 13 дек – РИА Новости. Правоохранители не обнаружили взрывное устройство в вагонах пассажирского поезда сообщением Москва — Киев на Киевском вокзале в центре Москвы после анонимного звонка ...",incident,0
1760,"Boeing сократит выпуск лайнеров Boeing 777 из-за низкого спроса\n\nИз-за снижения спроса Boeing сократит производство лайнеров Boeing 777 с августа 2017 г. с нынешних 8,3 единицы в месяц до 5 един...",economics,0
1899,"НЬЮ-ЙОРК. 13 ДЕКАБРЯ. ИНТЕРФАКС — Самолет авиакомпании Lufthansa, следовавший из Хьюстона (штат Техас) во Франкфурт (Германия), был перенаправлен в нью-йоркский международный аэропорт имени Джона ...",incident,0
2212,КИЕВ. 13 декабря. УНН. Российские правоохранители не обнаружили взрывное устройство в вагонах пассажирского поезда сообщением Москва - Киев на Киевском вокзале в центре Москвы после анонимного зво...,incident,0


In [36]:
data[ data['cluster']==1 ].sample(10)

,text,tag,cluster
929,"Доллар и евро подешевели в преддверии выходных\n\n9 декабря 2016 в 12:59\n\nTUT.BY\n\nФото: Дмитрий Брушко, TUT.BY\n\nНа Белорусской валютно-фондовой бирже 9 декабря прошли очередные торги валютам...",economics,1
876,"Куда пойти в выходные в Минске, если денег мало или нет совсем\n\n2 декабря 2016 в 8:16\n\nАлександра Гавриченко, AFISHA.TUT.BY\n\nПервые зимние выходные хороши уже тем, что они наступили. И напол...",reclama,1
1152,"""Минчанка"" стала обладателем Кубка Беларуси по волейболу\n\n4 декабря 2016 в 20:11\n\nБЕЛТА\n\nВолейболистки «Минчанки» выиграли финальный матч розыгрыша Кубка Беларуси со счетом 3:0 (25:16, 25:10...",sport,1
3130,Тюменские известия (Тюмень) Новоселий в Югре будет больше На июльском заседании\nдепутаты окружного парламента заслушали информацию правительства о реализации\nпрограммы ?Улучшение жилищных услови...,realty,1
652,"Онлайн-голосование ""HR-персона 2016""\n\n5 декабря 2016 в 9:00\n\n*На правах рекламы\n\n""Премия HR-бренд"" — ежегодная независимая премия за наиболее успешную работу с репутацией компании как работо...",reclama,1
1156,"Белорусские гандболистки разгромом команды Косова завершили минскую квалификацию к ЧМ-2017\n\n4 декабря 2016 в 15:44\n\nSPORT.TUT.BY\n\nЖенская сборная Беларуси по гандболу, досрочно обеспечившая ...",sport,1
2869,"Новый кроссовер приедет в Россию в двух кузовных модификациях: стандартной и длиннобазной XLV, которая будет длиннее на 230мм. Для обех версий в России предусмотрен только один 1,6-литровый бензин...",auto,1
2873,Южнокорейская машиностроительная компания SsangYong объявила о своем возвращении на российский авторынок и пообещала привезти в РФ новый внедорожник – Tivoli.\n\n\n\nНе так давно ряд автодилеров с...,auto,1
1057,"Вырастут пенсии и цены на сигареты. Что меняется с 1 декабря\n\n30 ноября 2016 в 7:00\n\nFINANCE.TUT.BY\n\nУ пенсионеров появятся лишние примерно 14 рублей, по традиции подорожают сигареты, а забы...",economics,1
759,15 лет грозит минчанину за выращивание и сбыт конопли с приусадебного участка под столицей\n\n7 декабря 2016 в 13:20\n\nTUT.BY\n\nЗавершено расследование уголовного дела в отношении 43-летнего мин...,incident,1


In [37]:
data[ data['cluster']==2 ].sample(10)

,text,tag,cluster
501,Лавров положительно оценил новые предложения США по Алеппо\n\n3 декабря 2016 в 14:42\n\nРБК\n\nСергей Лавров. Фото: пресс-служба ООН\n\nНовые предложения США по урегулированию ситуации в Алеппо во...,politics,2
3181,Департамент межнациональных отношений министерства регионального развиия РФ\nКОМИИНФОРМ. Новости Представитель Минрегионразвития РФ Александр Журавский\nоткрыл семинар для журналистов в Коми Дирек...,politics,2
1597,"Ранее глава Хорватии заявила, что Россия активно влияет на события на Балканах, следствием чего может стать военный конфликт в регионе\n\nМосква, , 09:56 — REGNUM Глава МИД РФ Сергей Лавров заявил...",politics,2
937,"Макей предложил лидерам России, США, ЕС и Китая встретиться в Минске для откровенного разговора\n\n8 декабря 2016 в 17:36\n\nTUT.BY\n\nЛидерам России, США, ЕС, Китая пора встретиться для откровенн...",economics,2
1677,"Первый президент СССР Михаил Горбачев считает, что в границах бывшего Советского Союза может появиться новое Союзное государство. Об этом он заявил в интервью ТАСС, приуроченном к 25-й годовщине р...",politics,2
3114,"Республика Коми Онлайн. В Сыктывкаре стартовал семинар по этнической\nтематике для журналистов Северо-Запада. Он продлится до 29 июля под эгидой\nгильдии межэтнической журналистики России, минист...",politics,2
1374,"Ранее глава Хорватии заявила, что Россия активно влияет на события на Балканах, следствием чего может стать военных конфликт в регионе\r\nГлава МИД РФ Сергей Лавров заявил, что Москва готова обсуд...",politics,2
1844,"БЕЛГРАД, 13 дек – РИА Новости. Заявления о возможной российской угрозе странно слышать от хорватских властей, однако Москва готова обсудить аргументы Загреба, заявил глава МИД Сергей Лавров.\n\nРа...",politics,2
3085,Информационный портал Республики Коми (komipress.ru) (Сыктывкар) Активисты\nкоми народа обеспокоены ?похоронами? языка коренного этноса республики\nАктивисты коми народа выразили озабоченность выс...,social,2
411,"Процесс о госизмене. Генпрокурор Украины анонсировал скорое начало суда над Януковичем\n\n10 декабря 2016 в 7:58\n\nLenta.ru\n\nГенеральный прокурор Украины Юрий Луценко сообщил, что судебный проц...",politics,2


In [38]:
data[ data['cluster']==3 ].sample(10)

,text,tag,cluster
552,"Легковушка сбила шестилетнего ребенка на пешеходном переходе в Волковыске\n\n4 декабря 2016 в 11:33\n\nAUTO.TUT.BY\n\nЛегковушка на пешеходном переходе сбила шестилетнего ребенка в Волковыске, пиш...",incident,3
2272,Снежная буря стала основной причиной массового ДТП в Кемеровской области. Во вторник на участке дороги Кемерово–Ленинск-Кузнецкий столкнулись более восьми автомобилей. В результате аварии один чел...,incident,3
2330,"МОСКВА, 12 декабря. /ТАСС/. Сотрудница полиции, в состоянии алкогольного опьянения сбившая насмерть двух человек в подмосковном Серпухове, уволена из органов внутренних дел. Об этом сообщила ТАСС ...",incident,3
2269,"На трассе в Кузбассе восемь машин попали в ДТП из-за непогоды\n\nКЕМЕРОВО, 13 дек — РИА Новости. Движение на автомагистрали Кемерово — Ленинск-Кузнецкий перекрыто во вторник из-за нескольких ДТП н...",incident,3
563,В Витебске лоб в лоб столкнулись милицейский автомобиль и медпомощь\n\n2 декабря 2016 в 15:50\n\nAUTO.TUT.BY\n\nДТП произошло 2 декабря в первом часу дня возле Кировского моста в Витебске. Столкну...,incident,3
589,"В Минске под колесами Ford погиб пешеход\n\n30 ноября 2016 в 10:02\n\nAUTO.TUT.BY\n\nСмертельная авария произошла 29 ноября, в 21.25, на столичной улице Семашко. Об этом AUTO.TUT.BY рассказали в Г...",incident,3
2347,"Никаких телесных повреждений, свидетельствующих о насильственном характере наступления смерти, не обнаружено\n\nРОСТОВ-НА-ДОНУ, 13 декабря, РИА ФедералПресс. В Ростове-на-Дону произошла трагедия. ...",incident,3
2231,Трагедия произошла 5 ноября. Мальчику стало плохо после школьных соревнований.\n\nВ Красноярской школе на церемонии награждения победителей школьных соревнований по баскетболу 16-летнему мальчику ...,incident,3
2288,"Огонь охватил 350 квадратных метров.\n\nВечером 12 декабря 2016 года в Екатеринбурге загорелся банный комплекс ""Русские бани"", расположенный на улице Водонасосной. Огонь захватил 350 квадратных ме...",incident,3
2323,"Сотрудница межмуниципального управления МВД России «Серпуховское», находившаяся за рулем в пьяном виде, насмерть сбила двоих человек в Подмосковье.\n\nПо информации главного управления МВД по Моск...",incident,3


In [39]:
data[ data['cluster']==4 ].sample(10)

,text,tag,cluster
1413,"Российский лидер прибудет в аэропорт префектуры Ямагути (остров Хонсю) во второй половине дня 15 декабря\n\nТокио, , 09:21 — REGNUM МИД Японии объявил 13 декабря официальную программу визита в стр...",politics,4
1875,"Токио, 13 декабря. Власти Японии во вторник обнародовали программу визита президента России Владимира Путина, который должен посетить страну 15-16 декабря. Российский лидер прибудет в префектуру Я...",politics,4
2456,"МОСКВА, 13 декабря. /ТАСС/. Чудо-женщина (Wonder Woman), один из культовых персонажей вселенной комиксов издательства DC Comics, больше не будет почетным послом ООН по борьбе за права и возможност...",culture,4
1727,Одна из крупнейших японских кредитных организаций Sumitomo Mitsui и российский банк ВТБ планируют подписать договор о сотрудничестве. Документ предусматривает двустороннее представление клиентов д...,economics,4
1398,"В России завершён глобальный социологический прогноз, проведенный специалистами «Левада-центра», он неожиданно показал преданность большей части россиян Владимиру Владимировичу Путину. Об этом пиш...",politics,4
1874,"13 декабря 2016, 08:56\n\nМИД Японии официально объявил программу двухдневного пребывания в стране российского лидера Владимира Путина.\n\nПрезидент РФ во второй половине дня 15 декабря прибывает ...",politics,4
1397,"Почти три четверти россиян уверены, что президент Российской Федерации Владимир Путин хочет добиться повышения благосостояния населения. При этом 24 процента полагают, что главе российского госуда...",politics,4
1390,"Отвечая на вопрос о причинах доверия главе государства, 28% заявили, что Путин «успешно и достойно справляется с решением проблем страны», 29% отметили, что люди не видят, на кого еще можно положи...",politics,4
1402,"В ходе опроса россияне ответили, по каким причинам склонны доверять президенту РФ Владимиру Путину. По их мнению, он успешно справляется с решением проблем государства, а также стремится улучшить ...",politics,4
1851,"МОСКВА, 13 дек — РИА Новости. Президент России Владимир Путин в интервью японской телекомпании Nippon TV (NTV) прокомментировал высокий уровень доверия у населения.\n\n""Люди видят мое стремление к...",politics,4


In [40]:
data[ data['cluster']==9 ].sample(10)

,text,tag,cluster
2070,"ФФУ раздала награды лучшим игрокам, тренерам, арбитру и футбольной школе 2016 года на специальной церемонии «Футбольні зірки України».\n\nЯрмоленко, Вернидуб и Коваленко названы лучшими по итогам ...",sport,9
2405,Голливудская ассоциация иностранной прессы определила номинантов на получение премии «Золотой глобус»\n\nЛидером по числу заявленных категорий стал мюзикл Дэмьена Шазелла «Ла-Ла Ленд» с Райаном Го...,culture,9
2066,"Нападающий киевского ""Динамо"" Андрей Ярмоленко признан лучшим футболистом Украины 2016 года. Об этом сообщается на официальном сайте Федерации футбола Украины. Помимо 27-летнего форварда на эту на...",sport,9
2401,"Вручение премий ""Золотой глобус"" состоится 8 января 2017 года.\n\nНоминантов на премию ""Золотой глобус"" объявила Ассоциация международной кинопрессы в понедельник, 12 декабря.\n\nПо информации Асс...",culture,9
2498,"По версии EFA (Европейской Киноакадемии) фильм «Тони Эрдманн» заслужил звание лучшего европейского фильма года, пишет «Газета.ru» со ссылкой на сайт Академии.\n\nФильм-трагикомедия рассказывает о ...",culture,9
2419,"Американский фильм-мюзикл «Ла-Ла Ленд» получил премию Critics Choice Award в номинации «Лучший фильм». Кинокартину снял Демиен Шазелл, который был удостоен награды как лучший режиссер. Церемония в...",culture,9
205,"В Лос-Анджелесе объявлены номинации на премию ""Энни"" Номинации на\nамериканскую премию ""Энни"", присуждаемую за лучшие работы в области\nмультипликации, объявлены в Лос-Анджелесе. Безусловным лидер...",culture,9
2402,"НЬЮ-ЙОРК, 12 декабря. /Корр. ТАСС Алексей Качалин/. Фильм-трагикомедия ""Ла-Ла Ленд"" (La La Land) американского режиссера Дэмьена Шазелла лидирует по числу номинаций на премию ""Золотой глобус"". Эта...",culture,9
2069,"Федерация футбола Украины и Ассоциация футболистов-профессионалов Украины выбрали лучшего игрока и тренера\n\nПочетное звание лучшего футболиста года на церемонии ""Футбольные звезды Украины""-2016,...",sport,9
917,"Трейлеры недели: ""Мумия"" с Томом Крузом, Бондарчук в роли Деда Мороза и нестареющий Джеки Чан\n\n5 декабря 2016 в 9:25\n\nAFISHA.TUT.BY\n\nПосле нескольких недель с сочными подборками кинотрейлеро...",culture,9
